## FineTuning LLM with Model-As-Service

This sample shows how use create a standalone FineTuning job to fine tune a model to summarize a dialog between 2 people using samsum dataset.

#### Training data
We use sample data placed along with the notebook for our finetuning in files "train.jsonl" and "validation.jsonl".

#### Model
We will use the Phi-3-mini-4k-instruct model to show how user can finetune a model for chat-completion task. If you opened this notebook from a specific model card, remember to replace the specific model name. 

#### Outline
1. Setup pre-requisites
2. Pick a model to fine-tune.
3. Create training and validation datasets.
4. Configure the fine tuning job.
5. Submit the fine tuning job.
6. Create serverless deployment using finetuned model and sample inference

### 1. Setup pre-requisites
* Install dependencies
* Connect to AzureML Workspace. Learn more at [set up SDK authentication](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication?tabs=sdk). Replace  `<WORKSPACE_NAME>`, `<RESOURCE_GROUP>` and `<SUBSCRIPTION_ID>` below.
* Connect to `azureml` system registry
* Set an optional experiment name

**Install dependencies by running below cell. This is not an optional step if running in a new environment.**

In [ ]:
%pip install azure-ai-ml
%pip install azure-identity

%pip install mlflow
%pip install azureml-mlflow

### Create AzureML Workspace connections

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import (
    DefaultAzureCredential,
    InteractiveBrowserCredential,
)

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

try:
    workspace_ml_client = MLClient.from_config(credential=credential)
except:
    workspace_ml_client = MLClient(
        credential,
        subscription_id="<SUBSCRIPTION_ID>",
        resource_group_name="<RESOURCE_GROUP_NAME>",
        workspace_name="<PROJECT_NAME OR WORKSPACE_NAME>",
    )

# the models, fine tuning pipelines and environments are available in various AzureML system registries,
# Example: Phi family of models are in "azureml", Llama family of models are in "azureml-meta" registry.
registry_ml_client = MLClient(credential, registry_name="azureml")

# Get AzureML workspace object.
workspace = workspace_ml_client._workspaces.get(workspace_ml_client.workspace_name)
workspace.id

### 2. Pick a model to fine tune

Phi-4-mini-instruct is a lightweight open model built upon synthetic data and filtered publicly available websites - with a focus on high-quality, reasoning dense data. The model belongs to the Phi-4 model family and supports 128K token context length. The model underwent an enhancement process, incorporating both supervised fine-tuning and direct preference optimization to support precise instruction adherence and robust safety measures. You can browse these models in the Model Catalog in the Azure AI Studio, filtering by the `chat-completion` task. In this example, we use the `Phi-3-mini-4k-instruct` model. If you have opened this notebook for a different model, replace the model name and version accordingly.

Note the model id property of the model. This will be passed as input to the fine tuning job. This is also available as the `Asset ID` field in model details page in Azure AI Studio Model Catalog.

In [ ]:
model_name = "Phi-4-mini-instruct"
model_to_finetune = registry_ml_client.models.get(model_name, label="latest")
print(
    "\n\nUsing model name: {0}, version: {1}, id: {2} for fine tuning".format(
        model_to_finetune.name, model_to_finetune.version, model_to_finetune.id
    )
)

### 3. Sample data
The chat-completion dataset entry using the following schema:


    {
        "prompt": "Create a fully-developed protagonist who is challenged to survive within a dystopian society under the rule of a tyrant. ...",
        "messages":[",
            {",
                "content": "Create a fully-developed protagonist who is challenged to survive within a dystopian society under the rule of a tyrant. ...",
                "role": "user",
            },
            {",
                "content": "Name: Ava\n Ava was just 16 years old when the world as she knew it came crashing down. The government had collapsed, leaving behind a chaotic and lawless society. ...",
                "role": "assistant",
            },
            {",
                "content": "Wow, Ava's story is so intense and inspiring! Can you provide me with more details.  ...",
                "role": "user",
            },
            {
                "content": "Certainly! ....",
                "role": "assistant"",
            }
        ],
        "prompt_id": "d938b65dfe31f05f80eb8572964c6673eddbd68eff3db6bd234d7f1e3b86c2af",
    }

#### Create data inputs

##### Training data

In [ ]:
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.entities import Data

dataset_version = "1"
train_dataset_name = "chat_training_small"
try:
    train_data_asset = workspace_ml_client.data.get(
        train_dataset_name, version=dataset_version
    )
    print(f"Dataset {train_dataset_name} already exists")
except:
    print("creating dataset")
    train_data = Data(
        path=f"./train.jsonl",
        type=AssetTypes.URI_FILE,
        description="Training dataset",
        name=train_dataset_name,
        version="1",
    )
    train_data_asset = workspace_ml_client.data.create_or_update(train_data)

##### Validation data (Optional)

In [ ]:
from azure.ai.ml.entities import Data

dataset_version = "1"
validation_dataset_name = "chat_validation_small"
try:
    validation_data_asset = workspace_ml_client.data.get(
        validation_dataset_name, version=dataset_version
    )
    print(f"Dataset {validation_dataset_name} already exists")
except:
    print("creating dataset")
    validation_data = Data(
        path=f"./validation.jsonl",
        type=AssetTypes.URI_FILE,
        description="Validation dataset",
        name=validation_dataset_name,
        version="1",
    )
    validation_data_asset = workspace_ml_client.data.create_or_update(validation_data)

#### Create marketplace subscription for 3P models
**Note:** Skip this step for 1P(Microsoft) models that are offered on Azure. Example: Phi family of models

In [ ]:
model_id_to_subscribe = "/".join(model_to_finetune.id.split("/")[:-2])
print(model_id_to_subscribe)

normalized_model_name = model_name.replace(".", "-")

In [ ]:
from azure.ai.ml.entities import MarketplaceSubscription


subscription_name = f"{normalized_model_name}-sub"

marketplace_subscription = MarketplaceSubscription(
    model_id=model_id_to_subscribe,
    name=subscription_name,
)

# note: this will throw exception if the subscription already exists or subscription is not required (for example, if the model is not in the marketplace like Phi family)
try:
    marketplace_subscription = (
        workspace_ml_client.marketplace_subscriptions.begin_create_or_update(
            marketplace_subscription
        ).result()
    )
except Exception as ex:
    print(ex)

### 3. Submit the fine tuning job using the the model and data as inputs
 
Create FineTuning job using all the data that we have so far.

#### Define finetune parameters

##### There are following set of parameters that are required.

1. `model` - Base model to finetune.
2. `training_data` - Training data for finetuning the base model.
3. `validation_data` - Validation data for finetuning the base model.
4. `task` - FineTuning task to perform. eg. CHAT_COMPLETION for chat-completion finetuning jobs.
5. `outputs`- Output registered model name.

##### Following parameters are optional:

1. `hyperparameters` - Parameters that control the FineTuning behavior at runtime.
2. `name`- FineTuning job name
3. `experiment_name` - Experiment name for FineTuning job.
4. `display_name` - FineTuning job display name.

In [ ]:
from azure.ai.ml.finetuning import FineTuningTaskType, create_finetuning_job
import uuid

guid = uuid.uuid4()
short_guid = str(guid)[:8]
display_name = f"{model_name}-display-name-{short_guid}-from-sdk"
name = f"{model_name}t-{short_guid}-from-sdk"
output_model_name_prefix = f"{model_name}-{short_guid}-from-sdk-finetuned"
experiment_name = f"{model_name}-from-sdk"

finetuning_job = create_finetuning_job(
    task=FineTuningTaskType.CHAT_COMPLETION,
    training_data=train_data_asset.id,
    validation_data=validation_data_asset.id,
    hyperparameters={
        "per_device_train_batch_size": "1",
        "learning_rate": "0.00002",
        "num_train_epochs": "1",
    },
    model=model_to_finetune.id,
    display_name=display_name,
    name=name,
    experiment_name=experiment_name,
    tags={"foo_tag": "bar"},
    properties={"my_property": "my_value"},
    output_model_name_prefix=output_model_name_prefix,
)

In [ ]:
created_job = workspace_ml_client.jobs.create_or_update(finetuning_job)
workspace_ml_client.jobs.get(created_job.name)

#### Wait for the above job to complete successfully

In [ ]:
status = workspace_ml_client.jobs.get(created_job.name).status

import time

while True:
    status = workspace_ml_client.jobs.get(created_job.name).status
    print(f"Current job status: {status}")
    if status in ["Failed", "Completed", "Canceled"]:
        print("Job has finished with status: {0}".format(status))
        break
    else:
        print("Job is still running. Checking again in 30 seconds.")
        time.sleep(30)

In [ ]:
finetune_model_name = created_job.outputs["registered_model"]["name"]
finetune_model_name

In [ ]:
# Deploy the model as a serverless endpoint

endpoint_name = f"{normalized_model_name}-ft-{short_guid}"  # Name must be unique
model_id = f"azureml://locations/{workspace.location}/workspaces/{workspace._workspace_id}/models/{finetune_model_name}/versions/1"

#### 4. Create serverless endpoint using the finetuned model


##### Option 1: Create serverless endpoint in the same project where it was finetuned

In [ ]:
from azure.ai.ml.entities import ServerlessEndpoint

serverless_endpoint = ServerlessEndpoint(name=endpoint_name, model_id=model_id)

created_endpoint = workspace_ml_client.serverless_endpoints.begin_create_or_update(
    serverless_endpoint
).result()

##### Option 2: Create Serverless endpoint in a different Region/Subscription/Project

 **Scenario**: User wants to create deployment in a different project/region/subscription under same tenant and avoid retraining.

**Current support matrix for FT deployments:** 

| Supported Models          | eastus2 | eastus | northcentralus | westus3 | westus | southcentralus | swedencentral |
|---------------------------|---------|--------|----------------|---------|--------|----------------|---------------|
| Mistral-3B               | ✅      | ✅     | ✅             | ✅      | ✅     | ✅             | ❌            |
| Mistral-Nemo             | ✅      | ✅     | ✅             | ✅      | ✅     | ✅             | ❌            |
| Mistral-Large-2411       | ✅      | ✅     | ✅             | ✅      | ✅     | ✅             | ❌            |
| tsuzumi-7b | ✅      | ✅     | ✅             | ✅      | ✅     | ✅              | ❌            |
| Phi-4-mini-instruct      | ✅      | ✅     | ✅             | ✅      | ✅     | ✅             | ❌            |
| Phi-3.5-mini-instruct    | ✅      | ✅     | ✅             | ✅      | ✅     | ✅             | ❌            |
| Phi-3.5-MOE-instruct     | ✅      | ✅     | ✅             | ✅      | ✅     | ✅             | ❌            |
| Phi-3-mini-4k-instruct   | ✅      | ❌      | ❌             | ❌      | ❌     | ❌            | ❌            |
| Phi-3-mini-128k-instruct   | ✅      | ❌      | ❌             | ❌      | ❌     | ❌      | ❌            |
| Phi-3-medium-4k-instruct | ✅      | ❌      | ❌             | ❌      | ❌     | ❌               | ❌            |
| Phi-3-medium-128k-instruct     | ✅      | ❌      | ❌             | ❌      | ❌     | ❌             | ✅               |
| Llama-3.1-8b-instruct | ❌     | ❌      | ❌             | ✅    | ❌     | ❌               | ❌            |
| Llama-3.1-70b-instruct | ❌     | ❌      | ❌             | ✅    | ❌     | ❌               | ❌            |
| Llama-2-7b | ❌     | ❌      | ❌             | ✅    | ❌     | ❌               | ❌            |
| Llama-2-13b | ❌     | ❌      | ❌             | ✅    | ❌     | ❌               | ❌            |
| Llama-2-70b | ❌     | ❌      | ❌             | ✅    | ❌     | ❌               | ❌            |


* Create MLClient with target Project

In [ ]:
# Create Cross region FT deployment client
from azure.ai.ml.entities import ServerlessEndpoint
from azure.ai.ml import MLClient
from azure.identity import (
    DefaultAzureCredential,
    InteractiveBrowserCredential,
)

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()
try:
    workspace_ml_client = MLClient.from_config(credential=credential)
except:
    workspace_ml_client = MLClient(
        credential,
        subscription_id="<TARGET_SUBSCRIPTION_ID>",
        resource_group_name="<TARGET_RESOURCE_GROUP_NAME>",
        workspace_name="<TARGET_PROJECT_NAME>",
    )

workspace = workspace_ml_client._workspaces.get(workspace_ml_client.workspace_name)

* Create marketplace subscription in the target project (SKIP this step for Microsoft 1P models like Phi-family)

In [ ]:
from azure.ai.ml.entities import MarketplaceSubscription


subscription_name = f"{normalized_model_name}-sub"

marketplace_subscription = MarketplaceSubscription(
    model_id=model_id_to_subscribe,
    name=subscription_name,
)

# note: this will throw exception if the subscription already exists or subscription is not required (for example, if the model is not in the marketplace like Phi family)
try:
    marketplace_subscription = (
        workspace_ml_client.marketplace_subscriptions.begin_create_or_update(
            marketplace_subscription
        ).result()
    )
except Exception as ex:
    print(str(ex))

* Create deployment in the target region/subscription/project

In [ ]:
workspace_region = workspace.location
model_to_finetune.tags
supported_regions = model_to_finetune.tags["maas-finetuning-deploy-regions"]
supported_regions
if workspace_region in supported_regions:
    print(f"Creating endpoint in the region:{workspace_region}")
    serverless_endpoint = ServerlessEndpoint(name=endpoint_name, model_id=model_id)
    created_endpoint = workspace_ml_client.serverless_endpoints.begin_create_or_update(
        serverless_endpoint
    ).result()
else:
    raise ValueError(
        f"For the model : {model_to_finetune}, the target region: {workspace_region} is not supported for deployment, the supported regions: {supported_regions}"
    )

#### 5. Inference on the Finetuned Serverless Endpoint

In [ ]:
endpoint = workspace_ml_client.serverless_endpoints.get(endpoint_name)
endpoint_keys = workspace_ml_client.serverless_endpoints.get_keys(endpoint_name)
auth_key = endpoint_keys.primary_key

In [ ]:
import requests

url = f"{endpoint.scoring_uri}/v1/chat/completions"

payload = {
    "max_tokens": 1024,
    "messages": [
        {
            "content": "This script is great so far. Can you add more dialogue between Amanda and Thierry to build up their chemistry and connection?",
            "role": "user",
        }
    ],
}
headers = {"Content-Type": "application/json", "Authorization": f"{auth_key}"}

response = requests.post(url, json=payload, headers=headers)

response.json()

#### 6. Delete the Finetuned Serverless Endpoint

In [ ]:
workspace_ml_client.serverless_endpoints.begin_delete(endpoint_name).result()